Create a test sql database from titanic dataset.

https://python.langchain.com/docs/use_cases/sql/csv/

In [1]:
import pandas as pd
from pyprojroot import here

In [2]:
df = pd.read_csv("./data/titanic.csv")
print(df.shape)
print(df.columns.tolist())
display(df.head(3))

(887, 8)
['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Fare']


,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250


### **SQL**

Using SQL to interact with CSV data is the recommended approach because it is easier to limit permissions and sanitize queries than with arbitrary Python.

Most SQL databases make it easy to load a CSV file in as a table (DuckDB, SQLite, etc.). Once you’ve done this you can use all of the chain and agent-creating techniques outlined in the SQL use case guide. Here’s a quick example of how we might do this with SQLite:

In [4]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
db_path = str("./data/test_sqldb.db")
db_path = f"sqlite:///{db_path}"

engine = create_engine(db_path)
# df.to_sql("titanic", engine, index=False)
df.to_sql("titanic", engine, index=False)

887

For multiple csv files, we can create a sql with multiple tables:
```
df1.to_sql("csv1_name", engine, index=False)
df2.to_sql("csv2_name", engine, index=False)
```

In [5]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM titanic WHERE Age < 2;")

sqlite
['titanic']


"[(1, 2, 'Master. Alden Gates Caldwell', 'male', 0.83, 0, 2, 29.0), (0, 3, 'Master. Eino Viljami Panula', 'male', 1.0, 4, 1, 39.6875), (1, 3, 'Miss. Eleanor Ileen Johnson', 'female', 1.0, 1, 1, 11.1333), (1, 2, 'Master. Richard F Becker', 'male', 1.0, 2, 1, 39.0), (1, 1, 'Master. Hudson Trevor Allison', 'male', 0.92, 1, 2, 151.55), (1, 3, 'Miss. Maria Nakid', 'female', 1.0, 0, 2, 15.7417), (0, 3, 'Master. Sidney Leonard Goodwin', 'male', 1.0, 5, 2, 46.9), (1, 3, 'Miss. Helene Barbara Baclini', 'female', 0.75, 2, 1, 19.2583), (1, 3, 'Miss. Eugenie Baclini', 'female', 0.75, 2, 1, 19.2583), (1, 2, 'Master. Viljo Hamalainen', 'male', 0.67, 1, 1, 14.5), (1, 3, 'Master. Bertram Vere Dean', 'male', 1.0, 1, 2, 20.575), (1, 3, 'Master. Assad Alexander Thomas', 'male', 0.42, 0, 1, 8.5167), (1, 2, 'Master. Andre Mallet', 'male', 1.0, 0, 2, 37.0042), (1, 2, 'Master. George Sibley Richards', 'male', 0.83, 1, 1, 18.75)]"

In [6]:
db.run("SELECT AVG(Age) FROM titanic WHERE Survived==1")

'[(28.408391812865496,)]'

**Equivalent in Pandas**

In [7]:
df[df["Age"]<2]

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
77,1,2,Master. Alden Gates Caldwell,male,0.83,0,2,29.0000
163,0,3,Master. Eino Viljami Panula,male,1.00,4,1,39.6875
171,1,3,Miss. Eleanor Ileen Johnson,female,1.00,1,1,11.1333
182,1,2,Master. Richard F Becker,male,1.00,2,1,39.0000
303,1,1,Master. Hudson Trevor Allison,male,0.92,1,2,151.5500
379,1,3,Miss. Maria Nakid,female,1.00,0,2,15.7417
384,0,3,Master. Sidney Leonard Goodwin,male,1.00,5,2,46.9000
466,1,3,Miss. Helene Barbara Baclini,female,0.75,2,1,19.2583
641,1,3,Miss. Eugenie Baclini,female,0.75,2,1,19.2583
751,1,2,Master. Viljo Hamalainen,male,0.67,1,1,14.5000


### **Create an agent to interact with the Database**

In [8]:
import os
# from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# from langchain_community.llms import Ollama
# local_llm = Ollama(
#     model="mistral:7b-instruct-q4_0",
#     # model="phi:2.7b",
#     temperature=0.7,  # Contrôle la créativité (0-1)
#     num_ctx=4096,     # Taille du contexte
#     top_p=0.9         # Diversité des réponses
# )

In [9]:
from langchain.chat_models import ChatOllama
local_llm = ChatOllama(
    model="llama3.1:8b-instruct-q4_0",
    temperature=0.7,
    num_ctx=4096,
    top_p=0.9
)

C:\Users\PC\AppData\Local\Temp\ipykernel_6100\887144165.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  local_llm = ChatOllama(


In [12]:
from langchain_community.agent_toolkits import create_sql_agent
agent_executor = create_sql_agent(llm=local_llm, db=db, agent_type="zero-shot-react-description", verbose=True, handle_parsing_errors=True)

In [13]:
print(local_llm.invoke("Hello").content)

Hello! How can I assist you today?


In [14]:
agent_executor.invoke({"input": "How many passengers was in the titanic?"})



> Entering new SQL Agent Executor chain...
Thought: I should look at the tables in the database to see what I can query.

Action: sql_db_list_tables
Action Input: titanicTo answer this question, we'll need to identify the most relevant table related to the Titanic.

Action: sql_db_schema
Action Input: titanic
CREATE TABLE titanic (
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"Siblings/Spouses Aboard" BIGINT, 
	"Parents/Children Aboard" BIGINT, 
	"Fare" FLOAT
)

/*
3 rows from titanic table:
Survived	Pclass	Name	Sex	Age	Siblings/Spouses Aboard	Parents/Children Aboard	Fare
0	3	Mr. Owen Harris Braund	male	22.0	1	0	7.25
1	1	Mrs. John Bradley (Florence Briggs Thayer) Cumings	female	38.0	1	0	71.2833
1	3	Miss. Laina Heikkinen	female	26.0	0	0	7.925
*/To answer this question, we'll need to identify the most relevant column related to the number of passengers.

Action: sql_db_schema
Action Input: titanic
CREATE TABLE titanic (
	"Survived" BIGINT, 
	"Pcl

{'input': 'How many passengers was in the titanic?',
 'output': '887\n\nExplanation:\nTo find the number of passengers in the Titanic, we first listed all tables in the database and identified that "titanic" was a relevant table. We then queried its schema to determine which columns were most relevant to answering the question. Since we needed to count the total number of rows (passengers), we used the COUNT(*) function and executed it using sql_db_query. The result was [(887,)], indicating that there were 887 passengers in the Titanic.'}

In [29]:
db.run("SELECT COUNT(*) FROM titanic")

'[(887,)]'

In [16]:
# agent_executor.invoke({"input": "what's the average age of survivors"})

In [18]:
agent_executor.invoke({"input": "How many women were in the titatic?"})



> Entering new SQL Agent Executor chain...
Question: How many women were in the titatic?
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input:titanicQuestion: How many women were in the titatic?
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input:titanicQuestion: How many women were in the titatic?
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input:titanicIt seems like you're having a bit of a conversation with yourself.

Let's break it down:

1. You want to find out how many women were on the Titanic, so you need to identify relevant tables in the database.
2. The `sql_db_list_tables` tool can list all tables

{'input': 'How many women were in the titatic?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [ ]:
# Equivalence in Pandas
df[df["Survived"]==1]["Age"].mean()

28.408391812865496